# Fintech final project baseline 
-----
本 notebook 作為 fintech 金融科技導論的期末專題競賽 baseline 程式說明。

* [競賽連結](https://tbrain.trendmicro.com.tw/Competitions/Details/24)

首先會就資料格式以及處理說明，接續簡介模型訓練,最終預測結果並輸出目標格式。

Reminder: XGBoost 版本會影響 performance，請同學多注意。

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import time
import collections
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
intep_table = {
    'cycam_min':0,
    'cycam_max':51244000,
    'total_asset_min':0,
    'total_asset_max':73863211,
    'usgam_min':-3642727,
    'usgam_max':38777239
}

#(out_max-out_min)*input/(input_max-input_min)

def Interpolation(val, src, dst):
    input_min, input_max = intep_table[f'{src}_min'], intep_table[f'{src}_max']
    output_min, output_max = intep_table[f'{dst}_min'], intep_table[f'{dst}_max']
    return ((output_max-output_min)*val+input_max*output_min-input_min*output_max)/(input_max-input_min)

## 資料前處理
這邊針對訓練資料和測試的資料作整理。
baseline主要會使用到的csv檔案如下:
  - public_train_custinfo_full_hashed.csv: 包含主要要判斷的alert key對應的幾項參數，顧客id, 風險等級, 職業, 行內總資產和年齡。
  - train_x_alert_date: 作為訓練資料的alert key以及發生日期，共23906筆。
  - public_x_alert_date: 作為公開測試集的alert key，格式同上共1845筆。
  - train_y_answer: 訓練資料alert key對應最後是否SAR。
  - 預測的案件名單及提交檔案範例: 用於生成預測結果

除此之外，還會使用到顧客資訊當作訓練資料:
  - public_train_x_ccba_full_hashed.csv
  - public_train_x_cdtx0001_full_hashed.csv
  - public_train_x_dp_full_hashed.csv
  - public_train_x_remit1_full_hashed.csv

前處理的方式包含:
  - 從 alert key 檢索出顧客資訊
  - 對非數值 feature 做 label encoding
  - 從顧客資訊中挑選適合的 features 當作訓練資料，這裡挑選離 alert date 最近的一筆顧客資訊當作 features
  - 統計 training data 缺失值數量

In [3]:
def train_preprocess(data_dir):
    # declare csv path
    train_alert_date_csv = os.path.join(data_dir, 'train_x_alert_date.csv')
    cus_info_csv = os.path.join(data_dir, 'public_train_x_custinfo_full_hashed.csv')
    y_csv = os.path.join(data_dir, 'train_y_answer.csv')

    ccba_csv = os.path.join(data_dir, 'public_train_x_ccba_full_hashed.csv')
    cdtx_csv = os.path.join(data_dir, 'public_train_x_cdtx0001_full_hashed.csv')
    dp_csv = os.path.join(data_dir, 'public_train_x_dp_full_hashed.csv')
    remit_csv = os.path.join(data_dir, 'public_train_x_remit1_full_hashed.csv')

    public_x_csv = os.path.join(data_dir, 'public_x_alert_date.csv')

    cus_csv = [ccba_csv, cdtx_csv, dp_csv, remit_csv]
    date_col = ['byymm', 'date', 'tx_date', 'trans_date']
    data_use_col = [[1,3,4,7,8],[2,3,4],[1,4,5,7],[3]]
    
    print('Reading csv...')
    # read csv
    df_y = pd.read_csv(y_csv)
    df_cus_info = pd.read_csv(cus_info_csv)
    df_date = pd.read_csv(train_alert_date_csv)
    cus_data = [pd.read_csv(_x) for _x in cus_csv]
    df_public_x = pd.read_csv(public_x_csv)

    # do label encoding
    le = LabelEncoder()
    cus_data[2].debit_credit = le.fit_transform(cus_data[2].debit_credit)
    
    cnts = [0] * 4
    labels = []
    training_data = []

    print('Start processing training data...')
    start = time.time()
    for i in range(df_y.shape[0]):
        # from alert key to get customer information
        cur_data = df_y.iloc[i]
        alert_key, label = cur_data['alert_key'], cur_data['sar_flag']

        cus_info = df_cus_info[df_cus_info['alert_key']==alert_key].iloc[0]
        cus_id = cus_info['cust_id']
        cus_features = cus_info.values[2:]

        date = df_date[df_date['alert_key']==alert_key].iloc[0]['date']


        cnt = 0
        for item, df in enumerate(cus_data):
            cus_additional_info = df[df['cust_id']==cus_id]
            cus_additional_info = cus_additional_info[cus_additional_info[date_col[item]]<=date]

            if cus_additional_info.empty:
                cnts[item] += 1
                len_item = len(data_use_col[item])
                if item == 2:
                    len_item -= 1
                cus_features = np.concatenate((cus_features, [np.nan] * len_item), axis=0)
            else:
                cur_cus_feature = cus_additional_info.loc[cus_additional_info[date_col[item]].idxmax()]
                
                cur_cus_feature = cur_cus_feature.values[data_use_col[item]]
                # 處理 實際金額 = 匯率*金額
                if item == 2:
                    cur_cus_feature = np.concatenate((cur_cus_feature[:2], [cur_cus_feature[2]*cur_cus_feature[3]], cur_cus_feature[4:]), axis=0)
                cus_features = np.concatenate((cus_features, cur_cus_feature), axis=0)
        labels.append(label)
        training_data.append(cus_features)
        # print(f"cus_id : {cus_id} features : {np.array(training_data)[-1]} train_shape : {np.array(training_data).shape}")
        print('\r processing data {}/{}'.format(i+1, df_y.shape[0]), end = '')
    print('Processing time: {:.3f} secs'.format(time.time()-start))
    print('Missing value of 4 csvs:', cnts)

    return np.array(training_data), labels

In [4]:
def test_preprocess(data_dir):
    # declare csv path
    cus_info_csv = os.path.join(data_dir, 'private_x_custinfo_full_hashed.csv')

    ccba_csv = os.path.join(data_dir, 'private_x_ccba_full_hashed.csv')
    cdtx_csv = os.path.join(data_dir, 'private_x_cdtx0001_full_hashed.csv')
    dp_csv = os.path.join(data_dir, 'private_x_dp_full_hashed.csv')
    remit_csv = os.path.join(data_dir, 'private_x_remit1_full_hashed.csv')

    private_x_csv = os.path.join(data_dir, 'private_x_alert_date.csv')

    cus_csv = [ccba_csv, cdtx_csv, dp_csv, remit_csv]
    date_col = ['byymm', 'date', 'tx_date', 'trans_date']
    data_use_col = [[1,3,4,7,8],[2,3,4],[1,4,5,7],[3]]
    
    print('Reading csv...')
    # read csv
    df_cus_info = pd.read_csv(cus_info_csv)
    cus_data = [pd.read_csv(_x) for _x in cus_csv]
    df_private_x = pd.read_csv(private_x_csv)

    # do label encoding
    le = LabelEncoder()
    cus_data[2].debit_credit = le.fit_transform(cus_data[2].debit_credit)

    print('Start processing testing data')
    testing_data, testing_alert_key = [], []
    for i in range(df_private_x.shape[0]):
        # from alert key to get customer information
        cur_data = df_private_x.iloc[i]
        alert_key, date = cur_data['alert_key'], cur_data['date']

        cus_info = df_cus_info[df_cus_info['alert_key']==alert_key].iloc[0]
        cus_id = cus_info['cust_id']
        cus_features = cus_info.values[2:]

        for item, df in enumerate(cus_data):
            cus_additional_info = df[df['cust_id']==cus_id]
            cus_additional_info = cus_additional_info[cus_additional_info[date_col[item]]<=date]

            if cus_additional_info.empty:
                len_item = len(data_use_col[item])
                if item == 2:
                    len_item -= 1
                cus_features = np.concatenate((cus_features, [np.nan] * len_item), axis=0)
            else:
                cur_cus_feature = cus_additional_info.loc[cus_additional_info[date_col[item]].idxmax()]
                cur_cus_feature = cur_cus_feature.values[data_use_col[item]]
                # 處理 實際金額 = 匯率*金額
                if item == 2:
                    cur_cus_feature = np.concatenate((cur_cus_feature[:2], [cur_cus_feature[2]*cur_cus_feature[3]], cur_cus_feature[4:]), axis=0)
                cus_features = np.concatenate((cus_features, cur_cus_feature), axis=0)

        testing_data.append(cus_features)
        testing_alert_key.append(alert_key)

        # print(cus_features)
        print('\r processing data {}/{}'.format(i+1, df_private_x.shape[0]), end = '')
    return np.array(testing_data), testing_alert_key

# 訓練資料處理

In [5]:
public_data_dir = './public_data'
private_data_dir = './private_data'
# data preprocessing
raw_training_data, labels = train_preprocess(public_data_dir)
raw_testing_data, testing_alert_key = test_preprocess(private_data_dir)
#[cusinfo, ccba_csv, cdtx_csv, dp_csv, remit_csv]
#[
# risk_rank,occupation_code,total_asset,AGE,
# lupay,cycam,usgam,inamt,cucsm,
# country,cur_type,amt,
# debit_credit,tx_type,tx_amt,info_asset_code,
# trade_amount_usd
# ]
print(raw_training_data[0])
print(raw_training_data.shape, raw_testing_data.shape)

Reading csv...
Start processing training data...
 processing data 23906/23906Processing time: 3244.545 secs
Missing value of 4 csvs: [7214, 9577, 3086, 17719]
Reading csv...
Start processing testing data
 processing data 2005/2005[1 17.0 375576.0 4 85428.0 301224.0 154122.0 0.0 151434.0 134 47 673.0 1 2
 309.0 nan]
(23906, 16) (2005, 16)


## 缺失值補漏Ours

In [6]:
for data in raw_training_data:
    #fill cycam
    total_asset = data[2]
    if np.isnan(data[5]):
        data[5] = Interpolation(total_asset, 'total_asset', 'cycam')
    #fill usgam
    if np.isnan(data[6]):
        data[6] = Interpolation(data[5], 'cycam', 'usgam')

    fill_usgam = [4,8,11]
    for idx in fill_usgam:
        if np.isnan(data[idx]):
            data[idx] = data[6] if idx != 11 else data[6] / 30
    
fill = [2,4,5,6,8,11]
for data in raw_training_data:
    print(data[fill])

[375576.0 85428.0 301224.0 154122.0 151434.0 673.0]
[2717416.0 -2082103.074134443 1885258.7589781333 -2082103.074134443
 -2082103.074134443 -69403.43580448143]
[326517.0 -3455206.8549791994 226527.34590701724 -3455206.8549791994
 -3455206.8549791994 -115173.56183263999]
[1014759.0 -3059946.456135017 704008.2537976856 -3059946.456135017
 -3059946.456135017 -101998.21520450056]
[241719.0 12565.0 150744.0 82748.0 12477.0 2758.266666666667]
[3634343.0 829364.0 7666339.0 2343836.0 781279.0 78127.86666666667]
[1022596.0 -3055445.6326663215 709445.3208106536 -3055445.6326663215
 -3055445.6326663215 -101848.18775554404]
[358135.0 0.0 175543.0 0.0 0.0 0.0]
[1759966.0 4421.0 131259.0 5795.0 4600.0 193.16666666666666]
[4076287.0 636.0 256134.0 3538.0 3410.0 114.0]
[13134.0 10706.0 50703.0 4590.0 4590.0 574.0]
[103230.0 3468207.0 130233.0 76602.0 3311863.0 108194.0]
[623999.0 71681.0 201968.0 22696.0 118443.0 756.5333333333333]
[82076.0 -3595590.3932606042 56941.77774101914 -3595590.3932606042
 -3

## 缺失值補漏
  可以發現有不少筆資料其實是有缺漏的，補上缺失值的方法有很多種，我們對於數值類資料補上中位數，對於類別類資料補上眾數。

In [8]:
''' Missing Value Imputation '''
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_zero = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
# for numerical index we do imputation using median
numerical_index = [0,2,3,4,5,6,7,8,11,14,15]
# Otherwise we select the most frequent
non_numerical_index = [9,10,12,13]

zero_index = [1]

train_numerical_data = raw_training_data[:, numerical_index]
train_non_numerical_data = raw_training_data[:, non_numerical_index]
train_zero_data = raw_training_data[:, zero_index]

imp_median.fit(train_numerical_data)
train_numerical_data = imp_median.transform(train_numerical_data)

imp_most_frequent.fit(train_non_numerical_data)
train_non_numerical_data = imp_most_frequent.transform(train_non_numerical_data)

imp_zero.fit(train_zero_data)
train_zero_data = imp_zero.transform(train_zero_data)

training_data_befonehot = np.concatenate((train_non_numerical_data, train_numerical_data, train_zero_data), axis=1)

  此外，若類別類資料跟數字大小沒關係，我們採用 one-hot encoding 將其編碼。

In [9]:
# for some catogorical features, we do one hot encoding
one_hot_index = [0,1,2,3]
onehotencorder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), one_hot_index)],
    remainder='passthrough'                     
)
onehotencorder.fit(training_data_befonehot)
training_data = onehotencorder.transform(training_data_befonehot)
print(training_data.shape)

(23906, 92)


# XGBoost 訓練

In [25]:

import xgboost as xgb
# 建立 XGBClassifier 模型
xgbrModel=xgb.XGBClassifier(random_state=0, n_estimators=1000, max_depth=8, 
learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', gpu_id=0)
# 使用訓練資料訓練模型
xgbrModel.fit(training_data, labels)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

# 預測與結果輸出
  利用訓練好的模型對目標alert key預測報SAR的機率以及輸出為目標格式。
  目標輸出筆數3850，其中public筆數為1845筆。
  因上傳格式需要private跟public alert key皆考慮，直接從預測範本統計要預測的alert key，預測結果輸出為prediction.csv。

In [26]:
# Do missing value imputation and one-hot encoding for testing data
test_numerical_data = raw_testing_data[:, numerical_index]
test_non_numerical_data = raw_testing_data[:, non_numerical_index]
test_zero_data = raw_testing_data[:, zero_index]

test_numerical_data = imp_median.transform(test_numerical_data)

test_non_numerical_data = imp_most_frequent.transform(test_non_numerical_data)

test_zero_data = imp_zero.transform(test_zero_data)

testing_data = np.concatenate((test_non_numerical_data, test_numerical_data, test_zero_data), axis=1)
testing_data = onehotencorder.transform(testing_data)

In [27]:
# Read csv of all alert keys need to be predicted
public_private_test_csv = os.path.join(private_data_dir, 'private_x_alert_date.csv')
df_public_private_test = pd.read_csv(public_private_test_csv)

# Predict probability
predicted = []
for i, _x in enumerate(xgbrModel.predict_proba(testing_data)):
    predicted.append([testing_alert_key[i], _x[1]])
predicted = sorted(predicted, reverse=True, key= lambda s: s[1])

# 考慮private alert key部分，滿足上傳條件
public_private_alert_key = df_public_private_test['alert_key'].values
print(len(public_private_alert_key))

# For alert key not in public, add zeros
for key in public_private_alert_key:
    if key not in testing_alert_key:
        predicted.append([key, 0])

predict_alert_key, predict_probability = [], []
for key, prob in predicted:
    predict_alert_key.append(key)
    predict_probability.append(prob)

df_predicted = pd.DataFrame({
    "alert_key": predict_alert_key,
    "probability": predict_probability
})

df_predicted.to_csv('prediction_private.csv', index=False)

2005


In [ ]:
import pickle
file_name = "xgb_reg2.pkl"

# save
pickle.dump(xgbrModel, open(file_name, "wb"))

# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))
print(xgb_model_loaded)